In [483]:
import os
from pathlib import Path
import comtypes.client
import tkinter
import pyautogui as pag
import time
from utils.helper import connect_to_sap2000, get_node_distance, get_lp, click_image, click_image_offset, maximize_window_temporarily, make_icon_finder
from utils.excel_helpers import open_excel, save_excel, insert_empty_column_before, get_cell_value, copy_columns_cde, paste_from_clipboard_to_excel
pag.FAILSAFE = True
pag.PAUSE = 0.5
time.sleep(1)

In [484]:
#Input cell
db, wb_name = get_cell_value("L2")
# db=1.41
mat_name = "Long Rebar"
# mat_name = "Longitudinal"
ICONS = Path("icons-unr")
print("db:", db)
print("Workbook name:", wb_name)

db: 1.0
Workbook name: Goodnight-2015-T30-Data.xlsx


In [485]:
icon = make_icon_finder(ICONS)
icon("define-button.PNG")

'icons-unr\\define-button.PNG'

In [486]:
sap_model = connect_to_sap2000()

if sap_model.GetModelIsLocked():
    sap_model.SetModelIsLocked(False)
    print("✓ Model unlocked")

ret = sap_model.SelectObj.All()
if not ret: print("✓ Column selected")


✓ Connected to SAP2000
✓ Column selected


In [487]:
try:
    axial_load = sap_model.PointObj.GetLoadForce("2")[7][1]
except:
    axial_load = 0
print("Axial Load:", axial_load)

Axial Load: -206.0


In [488]:
sec_name = sap_model.PropFrame.GetNameList()[1][0]
I_gross = sap_model.PropFrame.GetSectProps(sec_name)[5]
print("MOI:", I_gross)

MOI: 5087.157607159083


In [489]:
L = get_node_distance(sap_model, 1, 2)
# db = sap_model.PropFrame.SDShape.GetSolidCircle(sec_name, "Circle1")[10]
# if db in ['None', None, '']: print("Input db manually")
mat_name_list = sap_model.PropMaterial.GetNameList()[1]
if mat_name not in mat_name_list:
    raise ValueError(f"Material '{mat_name}' not found in the model. Available materials: {mat_name_list}")
fye = sap_model.PropMaterial.GetORebar_1(mat_name)[2]

In [490]:
L, db, fye

(96.0, 1.0, 68.0)

In [491]:
hinge_length = round(get_lp(L, db, fye), 3)
print("Hinge Length:", hinge_length)

Hinge Length: 20.4


In [492]:
click_image(icon("sap-2000.PNG"))
pag.press("alt")
pag.press("d")
pag.press("p")
pag.press("f")

✓ Clicked icons-unr\sap-2000.PNG at (35, 10)


In [493]:
# click_image(icon("define-button.PNG"))
# click_image(icon("define-button.PNG"))
# click_image(icon("section-properties.png"))
# click_image(icon("frame-sections.png"))
click_image(icon("modify-property.PNG"))
click_image(icon("section-designer.PNG"))
time.sleep(4)
click_image(icon("display-SD.PNG"))
click_image(icon("show-m-phi.png"))

✓ Clicked icons-unr\modify-property.PNG at (1089, 472)
✓ Clicked icons-unr\section-designer.PNG at (959, 645)
✓ Clicked icons-unr\display-SD.PNG at (556, 178)
✓ Clicked icons-unr\show-m-phi.png at (651, 265)


True

In [494]:
time.sleep(1)
click_image_offset(icon("idealized.PNG"), x_offset_ratio=0.1)
click_image_offset(icon("axial.PNG"), x_offset_ratio=0.8)
pag.press('backspace', presses=5)
pag.write(f"{axial_load}")
click_image_offset(icon("no-of-points.PNG"), x_offset_ratio=1.3)
pag.press('backspace', presses=5)
pag.write('100')
click_image_offset(icon("details-m-phi.PNG"))
maximize_window_temporarily()
time.sleep(1)
click_image_offset(icon("I_crack.PNG"), x_offset_ratio=1)
pag.drag(80, duration=0.5)
pag.hotkey('ctrl', 'c')
I_crack = float(tkinter.Tk().clipboard_get())
print("I_crack=", I_crack )
click_image_offset(icon("ok.PNG"))
click_image_offset(icon("done.PNG"))
time.sleep(1.5)
click_image_offset(icon("done2.PNG"))
pag.press('esc')
pag.press('esc')

✓ Clicked icons-unr\idealized.PNG at (531, 676) [x_ratio=0.1, y_ratio=0.5, clicks=1]
✓ Clicked icons-unr\axial.PNG at (677, 706) [x_ratio=0.8, y_ratio=0.5, clicks=1]
✓ Clicked icons-unr\no-of-points.PNG at (846, 679) [x_ratio=1.3, y_ratio=0.5, clicks=1]
✓ Clicked icons-unr\details-m-phi.PNG at (992, 827) [x_ratio=0.5, y_ratio=0.5, clicks=1]
✓ Clicked icons-unr\I_crack.PNG at (72, 575) [x_ratio=1, y_ratio=0.5, clicks=1]
I_crack= 2890.8037
✓ Clicked icons-unr\ok.PNG at (960, 990) [x_ratio=0.5, y_ratio=0.5, clicks=1]
✓ Clicked icons-unr\done.PNG at (1135, 858) [x_ratio=0.5, y_ratio=0.5, clicks=1]
✓ Clicked icons-unr\done2.PNG at (1593, 923) [x_ratio=0.5, y_ratio=0.5, clicks=1]


In [495]:
k_mod = round(I_crack / I_gross, 5)
sap_model.PropFrame.SetModifiers(sec_name, [1, 1, 1, 1, k_mod, k_mod, 1, 1])

[(1.0, 1.0, 1.0, 1.0, 0.56826, 0.56826, 1.0, 1.0), 0]

In [496]:
ret = sap_model.SelectObj.All()
click_image_offset(icon("assign.PNG"), n=1)
click_image_offset(icon("assign-frame.PNG"))
click_image_offset(icon("hinges.PNG"))
with pag.hold('ctrl'):
    click_image_offset(icon("modify-hinge.PNG"))
time.sleep(1)
click_image_offset(icon("hinge-length.PNG"), x_offset_ratio=1.4)
pag.press('backspace', presses=7)
pag.write(f"{hinge_length}")
time.sleep(1)
click_image_offset(icon("ok-hinge-length.PNG"))
click_image_offset(icon("ok-hinge.PNG"))
time.sleep(2)

✓ Clicked icons-unr\assign.PNG at (303, 36) [x_ratio=0.5, y_ratio=0.5, clicks=1]
✓ Clicked icons-unr\assign-frame.PNG at (325, 88) [x_ratio=0.5, y_ratio=0.5, clicks=1]
✓ Clicked icons-unr\hinges.PNG at (619, 432) [x_ratio=0.5, y_ratio=0.5, clicks=1]
✓ Clicked icons-unr\modify-hinge.PNG at (1257, 303) [x_ratio=0.5, y_ratio=0.5, clicks=1]
✓ Clicked icons-unr\hinge-length.PNG at (1217, 338) [x_ratio=1.4, y_ratio=0.5, clicks=1]
✓ Clicked icons-unr\ok-hinge-length.PNG at (897, 548) [x_ratio=0.5, y_ratio=0.5, clicks=1]
✓ Clicked icons-unr\ok-hinge.PNG at (885, 722) [x_ratio=0.5, y_ratio=0.5, clicks=1]


In [497]:
sap_model.Analyze.RunAnalysis()
time.sleep(1)
pag.hotkey('alt')
pag.press('p')
pag.press('C')
# click_image_offset(os.path.join("icons", "display.PNG"), n=2)
# click_image_offset(os.path.join("icons", "pushover.PNG"))
pag.hotkey('shift', 'f12')
# pag.hotkey('alt', 'f')
# pag.press('t')
# pag.press('e')
click_image_offset(icon("select-table.PNG"), x_offset_ratio=0.2)
pag.hotkey('ctrl', 'c')

✓ Clicked icons-unr\select-table.PNG at (527, 364) [x_ratio=0.2, y_ratio=0.5, clicks=1]


In [498]:
from utils.excel_helpers import read_sap_clipboard_and_trim
df_trimmed = read_sap_clipboard_and_trim(L)

In [499]:
time.sleep(2)
pag.hotkey('alt')
pag.press('f')
pag.press('C', 2)
pag.press('Enter')
pag.press('Esc')


In [500]:
model_path = sap_model.GetModelFilepath()
model_name = Path(sap_model.GetModelFilename()).stem.lower()

In [501]:

if any(key in model_name for key in ["aashto", "ashto", "ashtoh", "asht ho", "ashtoh", "ashtto", "ashtho", "aashtho"]):
    save_as_name = "pushover-AASHTO.csv"
else:
    save_as_name = "pushover.csv"
df_trimmed.to_csv(Path(model_path) / save_as_name, index=False)
print(f"✓ Saved pushover results to {save_as_name}")

✓ Saved pushover results to pushover-AASHTO.csv


In [502]:
open_excel(Path(model_path) / save_as_name)

Workbook "C:\Users\abinashm\Box\AASHTO RC Column Design Evaluation V2\Abinash-Analysis_fibers not updated_SAP updated\Circular\(Goodnight 2015)\T30\sap2000\pushover-AASHTO.csv" opened and brought to front.


In [503]:
insert_empty_column_before(save_as_name, "D")

Empty column inserted before column D in "pushover-AASHTO.csv".


In [504]:
copy_columns_cde(save_as_name)

Columns C, D, E from row 2 to 173 copied to clipboard from 'pushover-AASHTO.csv'.


In [505]:
paste_mode = "AASHTO" if save_as_name == "pushover-AASHTO.csv" else None
# paste_mode = None
paste_from_clipboard_to_excel(wb_name, paste_mode)

Clipboard pasted into 'Goodnight-2015-T30-Data.xlsx' at O4 and rows below cleared for 3 columns.
